# Подбор параметров

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import (
    ensemble,
    datasets,
    model_selection,
)
import pandas as pd
import numpy as np

Мы будем использовать данные Boston, содержащие информацию о ценах на объекты недвижимости в одноименном городе. Загрузим данные с помощью sklearn.datasets:

In [20]:
boston = datasets.load_boston()
features = boston['data']
labels = boston['target']

Создадим модель градиентного бустинга для регрессии:

In [22]:
model = ensemble.GradientBoostingRegressor()
model

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

Прежде всего, попробуем подобрать n_estimators, learning_rate и max_depth. Для этого создадим dict, в котором для каждого параметра укажем возможные значения:

In [23]:
parameters = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [10, 100, 200],
}

Создадим мета-модель GridSearchCV, которая перепробует все возможные конфигурации и найдет наилучшую. При подборе будем использовать кросс-валидацию с 5 фолдами (по 5 подвыборкам):

In [37]:
search = model_selection.GridSearchCV(model, parameters, cv=5)
search

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 5, 7, 9], 'learning_rate': [0.01, 0.1, 0.5], 'n_estimators': [10, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

Запустим обучение:

In [33]:
search.fit(features, labels)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 5, 7, 9], 'learning_rate': [0.01, 0.1, 0.5], 'n_estimators': [10, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

Выберем наилучший вариант:

In [35]:
best_model = search.best_estimator_

Выведем score (коэффициент детерминации R^2):

In [38]:
best_model.score(features, labels)

0.98864199972870437

Аналогичным образом мы можем использовать random search (RandomizedSearchCV), который перебирает параметры случайным образом n_iter раз:

In [43]:
random_search = model_selection.RandomizedSearchCV(model, parameters, cv=5, n_iter=10).fit(features, labels)

Выведем наилучший "случайный" вариант:

In [44]:
random_search.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=200, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

и его R^2:

In [46]:
best_model.score(features, labels)

0.98864199972870437

Как правило, random search становится предпочтительным при тонкой настройке параметров и при большом их числе, когда grid search (перебор по сетке) становится неэффективным.